# Viola Jones Implementation from Scratch

Data available at : https://drive.google.com/drive/folders/1UBGUwLAqbtt03RtVhSMrD-AAOOWV3oLr?usp=sharing

In [1]:
import numpy as np
# from PIL import Image
import os
import cv2
from functools import partial
from multiprocessing import Pool

In [5]:
pos_training_path = '../face_data/small_data/face/pos_integral.npy'
neg_training_path = '../face_data/small_data/non/neg_integral.npy'

In [6]:
faces_int_img_training = np.load(pos_training_path)[:500]
non_faces_int_img_training = np.load(neg_training_path)[:500]

faces_int_img_test = np.load(pos_training_path)[800:900]
non_faces_int_img_test = np.load(neg_training_path)[800:900]

In [11]:
print(np.shape(faces_int_img_training))
print(np.shape(non_faces_int_img_training))

print(np.shape(faces_int_img_test))
print(np.shape(non_faces_int_img_test))

(500, 50, 50)
(500, 50, 50)
(100, 50, 50)
(100, 50, 50)


In [12]:
num_classifiers = 20
min_height = 4
max_height = 10
min_width = 4
max_width = 10

In [13]:
def enum(**enums):
    return type('Enum', (), enums)

FeatureType = enum(TWO_VERTICAL=(1, 2), TWO_HORIZONTAL=(2, 1), THREE_HORIZONTAL=(3, 1), 
                   THREE_VERTICAL=(1, 3), FOUR=(2, 2))
FeatureTypes = [FeatureType.TWO_VERTICAL, FeatureType.TWO_HORIZONTAL, FeatureType.THREE_VERTICAL, 
                FeatureType.THREE_HORIZONTAL, FeatureType.FOUR]

In [14]:
class HaarLikeFeature(object):
    """
    Class representing a haar-like feature.
    """

    def __init__(self, feature_type, position, width, height, threshold, polarity):
        """
        Creates a new haar-like feature.
        """
        self.type = feature_type
        self.top_left = position
        self.bottom_right = (position[0] + width, position[1] + height)
        self.width = width
        self.height = height
        self.threshold = threshold
        self.polarity = polarity
        self.weight = 1
    
#     def __str__(self):
#         return str(self.__class__) + ": " + str(self.__dict__)
    def get_score(self, int_img):
        """
        Get score for given integral image array.
        """
        score = 0
        if self.type == FeatureType.TWO_VERTICAL:
            first = sum_region(int_img, self.top_left, (self.top_left[0] + self.width, int(self.top_left[1] + self.height / 2)))
            second = sum_region(int_img, (self.top_left[0], int(self.top_left[1] + self.height / 2)), self.bottom_right)
            score = first - second
        elif self.type == FeatureType.TWO_HORIZONTAL:
            first = sum_region(int_img, self.top_left, (int(self.top_left[0] + self.width / 2), self.top_left[1] + self.height))
            second = sum_region(int_img, (int(self.top_left[0] + self.width / 2), self.top_left[1]), self.bottom_right)
            score = first - second
        elif self.type == FeatureType.THREE_HORIZONTAL:
            first = sum_region(int_img, self.top_left, (int(self.top_left[0] + self.width / 3), self.top_left[1] + self.height))
            second = sum_region(int_img, (int(self.top_left[0] + self.width / 3), self.top_left[1]), (int(self.top_left[0] + 2 * self.width / 3), self.top_left[1] + self.height))
            third = sum_region(int_img, (int(self.top_left[0] + 2 * self.width / 3), self.top_left[1]), self.bottom_right)
            score = first - second + third
        elif self.type == FeatureType.THREE_VERTICAL:
            first = sum_region(int_img, self.top_left, (self.bottom_right[0], int(self.top_left[1] + self.height / 3)))
            second = sum_region(int_img, (self.top_left[0], int(self.top_left[1] + self.height / 3)), (self.bottom_right[0], int(self.top_left[1] + 2 * self.height / 3)))
            third = sum_region(int_img, (self.top_left[0], int(self.top_left[1] + 2 * self.height / 3)), self.bottom_right)
            score = first - second + third
        elif self.type == FeatureType.FOUR:
            # top left area
            first = sum_region(int_img, self.top_left, (int(self.top_left[0] + self.width / 2), int(self.top_left[1] + self.height / 2)))
            # top right area
            second = sum_region(int_img, (int(self.top_left[0] + self.width / 2), self.top_left[1]), (self.bottom_right[0], int(self.top_left[1] + self.height / 2)))
            # bottom left area
            third = sum_region(int_img, (self.top_left[0], int(self.top_left[1] + self.height / 2)), (int(self.top_left[0] + self.width / 2), self.bottom_right[1]))
            # bottom right area
            fourth = sum_region(int_img, (int(self.top_left[0] + self.width / 2), int(self.top_left[1] + self.height / 2)), self.bottom_right)
            score = first - second - third + fourth
        return score
    
    def get_vote(self, int_img):
        """
        Get vote of this feature for given integral image.
        """
        score = self.get_score(int_img)
        return self.weight * (1 if score < self.polarity * self.threshold else -1)

In [15]:
def create_features(img_height, img_width, min_feature_width, max_feature_width, min_feature_height, max_feature_height):
    '''
    Runs multiple sized windows and creates features within the windows at each location 
    '''
    print('Creating haar-like features..')
    features = []
    print(FeatureTypes)
    for feature in FeatureTypes:
        # FeatureTypes are just tuples
        feature_start_width = max(min_feature_width, feature[0])
        for feature_width in range(feature_start_width, max_feature_width, feature[0]):
            feature_start_height = max(min_feature_height, feature[1])
            for feature_height in range(feature_start_height, max_feature_height, feature[1]):
                for x in range(img_width - feature_width):
                    for y in range(img_height - feature_height):
                        features.append(HaarLikeFeature(feature, (x, y), feature_width, feature_height, 0, 1))
                        features.append(HaarLikeFeature(feature, (x, y), feature_width, feature_height, 0, -1))
    print(str(len(features)) + ' features created.\n')
    return features

def get_feature_vote(feature, image):
    return feature.get_vote(image)

In [16]:
def sum_region(integral_img_arr, top_left, bottom_right):
    """
    Calculates the sum in the rectangle specified by the given tuples of features.
    """
    # swap tuples
    top_left = (top_left[1], top_left[0])
    bottom_right = (bottom_right[1], bottom_right[0])
    if top_left == bottom_right:
        return integral_img_arr[top_left]
    top_right = (bottom_right[0], top_left[1])
    bottom_left = (top_left[0], bottom_right[1])
    return integral_img_arr[bottom_right] - integral_img_arr[top_right] - integral_img_arr[bottom_left] + integral_img_arr[top_left]

In [17]:
def learn(pos_ii, neg_ii, num_classifiers=-1, min_width=1, max_width=-1, min_height=1, max_height=-1):
    '''
        the main learn function that creates classifiers, calculates 
        scores at various windows and locations and returns the classifiers
    '''
    num_pos = len(pos_ii)
    num_neg = len(neg_ii)
    num_imgs = num_pos + num_neg
    img_height, img_width = pos_ii[0].shape
    print(num_pos, num_neg, num_imgs, img_height, img_width)
    
    # set maximum feature heights and widths
    max_height = img_height if max_height == -1 else max_height
    max_width = img_width if max_width == -1 else max_width
#     print(max_height, max_width)

    # Create initial weights and labels
    pos_weights = np.ones(num_pos) * 1. / (2 * num_pos)
    neg_weights = np.ones(num_neg) * 1. / (2 * num_neg)
    weights = np.hstack((pos_weights, neg_weights))
    labels = np.hstack((np.ones(num_pos), np.ones(num_neg) * -1))
#     print(pos_weights, neg_weights, weights,labels)

    images = np.vstack((pos_ii, neg_ii))
    print(np.shape(images))

    features = create_features(img_height, img_width, min_width, max_width, min_height, max_height)
    num_features = len(features)
    feature_indexes = list(range(num_features))
    
    num_classifiers = num_features if num_classifiers == -1 else num_classifiers
    
    print('Calculating scores for images..')

    votes = np.zeros((num_imgs, num_features))
#     print(votes.shape)
    
    pool = Pool(processes=None)
    for i in range(num_imgs):
        votes[i] = np.array(list(pool.map(partial(get_feature_vote, image=images[i]), features)))
#         for j in range(num_features):
#             votes[i] = np.array(get_feature_vote(features[j], images[i]))
    
    classifiers = []

    print('Selecting classifiers..')
    
    for t in range(num_classifiers):

        classification_errors = np.zeros(len(feature_indexes))

        # normalize weights
        weights *= 1. / np.sum(weights)

        # select best classifier based on the weighted error
        for f in range(len(feature_indexes)):
            f_idx = feature_indexes[f]
            # classifier error is the sum of image weights where the classifier
            # is right
            error = sum(map(lambda img_idx: weights[img_idx] if labels[img_idx] != votes[img_idx, f_idx] else 0, range(num_imgs)))
            classification_errors[f] = error
            
        min_error_idx = np.argmin(classification_errors)
        best_error = classification_errors[min_error_idx]
        best_feature_idx = feature_indexes[min_error_idx]

        # set feature weight
        best_feature = features[best_feature_idx]
        feature_weight = 0.5 * np.log((1 - best_error) / best_error)
        best_feature.weight = feature_weight

        classifiers.append(best_feature)

        # update image weights
        weights = np.array(list(map(lambda img_idx: weights[img_idx] * np.sqrt((1-best_error)/best_error) if labels[img_idx] != votes[img_idx, best_feature_idx] else weights[img_idx] * np.sqrt(best_error/(1-best_error)), range(num_imgs))))

        # remove feature (a feature can't be selected twice)
        feature_indexes.remove(best_feature_idx)

    return classifiers

In [18]:
classifiers = learn(faces_int_img_training, non_faces_int_img_training, num_classifiers, min_height, max_height, min_width,
      max_width)

500 500 1000 50 50
(1000, 50, 50)
Creating haar-like features..
[(1, 2), (2, 1), (1, 3), (3, 1), (2, 2)]
..done. 265572 features created.

Calculating scores for images..
Selecting classifiers..


In [19]:
# print(votes[0][0:100])

In [20]:
def ensemble_vote(int_img, classifiers):
    """
    Classifies given integral image (numpy array) using given classifiers, 
    """
    return 1 if sum([c.get_vote(int_img) for c in classifiers]) >= 0 else 0


def ensemble_vote_all(int_imgs, classifiers):
    """
    Classifies given list of integral images (numpy arrays) using classifiers,
    
    """
    vote_partial = partial(ensemble_vote, classifiers=classifiers)
    return list(map(vote_partial, int_imgs))

In [21]:
#integral tests, face non face
print(np.shape(faces_int_img_test))Data available at : 

(100, 50, 50)


In [22]:
print('Testing selected classifiers..')
correct_faces = 0
correct_non_faces = 0
correct_faces = sum(ensemble_vote_all(faces_int_img_test, classifiers))
correct_non_faces = len(non_faces_int_img_test) - sum(ensemble_vote_all(non_faces_int_img_test, classifiers))
print(correct_faces, correct_non_faces, len(classifiers))
print('..done.\n\nResult:\n      Faces: ' + str(correct_faces) + '/' + str(len(faces_int_img_test))
      + '  (' + str((floati.e. if the sum of all classifier votes is greater 0, an image is classified
    positively (1) else negatively (0). The threshold is 0, because votes can be
    +1 or -1.
    :param int_imgs: List of integral images to be classified
    :type int_imgs: list[numpy.ndarray]
    :param classifiers: List of classifiers
    :type classifiers: list[violajones.HaarLikeFeature.HaarLikeFeature]
    :return: List of assigned labels, 1 if image was classified positively, else
    0
    :rtype: list[int](correct_faces) / len(faces_int_img_test)) * 100) + '%)\n  non-Faces: '
      + str(correct_non_faces) + '/' + str(len(non_faces_int_img_test)) + '  ('
      + str((float(correct_non_faces) / len(non_faces_int_img_test)) * 100) + '%)')

Testing selected classifiers..
82 70 20
..done.

Result:
      Faces: 82/100  (82.0%)
  non-Faces: 70/100  (70.0%)
